<a href="https://colab.research.google.com/github/perkarlsson/CE/blob/master/3LYNCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What needs to be coded (start with 2 entries):

1. Import a list of entires (being able to import a csv from Google Drive).

In [0]:
from google.colab import drive
import pandas as pd
drive.mount('drive')
mydateparser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d")
df = pd.read_csv("drive/My Drive/190831_3lynch_list.csv", header=0, parse_dates=['Date'], date_parser=mydateparser, encoding='utf-8-sig')
#df = pd.read_csv("drive/My Drive/input.csv", header=0, parse_dates=['Date'], date_parser=mydateparser)
#df=pd.read_csv("drive/My Drive/input.csv", header=0, parse_dates=True)
print(df.head())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive
        Date Ticker Setup Newsletter
0 2018-12-07   FLNT    BO        EOD
1 2018-12-07   OSIR    BO        EOD
2 2018-12-07   RDFN    BO        EOD
3 2018-12-10   IMMY    BO        EOD
4 2018-12-11    CHD    BO        EOD


2. For each entry pull down the financial data from the date of the entry to todays date (to begin with I will not need to store and open csvs).

In [0]:
import pandas_datareader as pdr
import datetime

ticker = {}

print("Acquiring breakout setups...")

for index, row in df.iterrows():
  if(df.iloc[index].Setup=="BO"):
    print("Acquiring BO setup " + str(row['Ticker']) + " for " + str(df.iloc[index].Date.date()))
    try: 
      ticker[index] = pdr.get_data_yahoo(row['Ticker'], start=df.iloc[index].Date-datetime.timedelta(90), end=df.iloc[index].Date+datetime.timedelta(90))
      ticker[index].reset_index(inplace=True)
    except: 
      print("Unable to acquire " + str(str(row['Ticker'])))
  else:
    continue
print("Finished acquiring breakout setups.")

Acquiring breakout setups...
Acquiring BO setup FLNT for 2018-12-07
Acquiring BO setup OSIR for 2018-12-07
Unable to acquire OSIR
Acquiring BO setup RDFN for 2018-12-07
Acquiring BO setup IMMY for 2018-12-10
Unable to acquire IMMY
Acquiring BO setup CHD for 2018-12-11
Acquiring BO setup MTLS for 2018-12-12
Acquiring BO setup NEW for 2018-12-12
Acquiring BO setup SSRM for 2018-12-12
Acquiring BO setup AEM for 2018-12-13
Acquiring BO setup AG for 2018-12-17
Acquiring BO setup AUY for 2018-12-17
Acquiring BO setup BVN for 2018-12-17
Acquiring BO setup CDE for 2018-12-17
Acquiring BO setup GDX for 2018-12-17
Acquiring BO setup HI for 2018-12-17
Acquiring BO setup IAG for 2018-12-17
Acquiring BO setup JNUG for 2018-12-17
Acquiring BO setup KGC for 2018-12-17
Acquiring BO setup PVG for 2018-12-17
Acquiring BO setup TMF for 2018-12-17
Acquiring BO setup IIIV for 2018-12-18
Acquiring BO setup RARX for 2018-12-18
Acquiring BO setup UFCS for 2018-12-21
Acquiring BO setup QTT for 2018-12-26
Acqui

In [0]:
#import plotly.figure_factory as ff
import plotly.graph_objects as go
from datetime import datetime

fig = {}
#hej
print("Plotting breakout charts..")
for ticker_index, row in df.iterrows():
  if(df.iloc[ticker_index].Setup=="BO"):
    try:
      #print("Daily graph of " + str(row['Ticker']))
      #fig[ticker_index] = ff.create_candlestick(ticker[ticker_index]['Open'], ticker[ticker_index]['High'], ticker[ticker_index]['Low'], ticker[ticker_index]['Close'], dates=ticker[ticker_index]['Date'])
      fig[ticker_index] = go.Figure(data=[go.Candlestick(x=ticker[ticker_index]['Date'], open=ticker[ticker_index]['Open'], high=ticker[ticker_index]['High'], low=ticker[ticker_index]['Low'], close=ticker[ticker_index]['Close'])])
      fig[ticker_index]['layout'].update({'title': str(row['Ticker']),'yaxis': {'title': 'Price ($)'}, 'shapes': [{
      'x0': df.iloc[ticker_index].Date.date(), 'x1': df.iloc[ticker_index].Date.date(),
      'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
      'line': {'color': 'rgb(30,30,30)', 'width': 1}
      }], })
      fig[ticker_index].update_layout(xaxis_rangeslider_visible=False)
      #fig['layout'].update({'title': str(row['Ticker']),'yaxis': {'title': 'Price ($)'},'shapes': [{'x0': df.iloc[index].Date.date(), 'x1': df.iloc[index].Date.date(),'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper','line': {'color': 'rgb(30,30,30)', 'width': 1}}],
      #  'annotations': [{'x': df.iloc[index].Date.date(), 'y': 0.05, 'xref': 'x', 'yref': 'paper','showarrow': True, 'xanchor': 'left','text': 'Buy signal'}]})
      fig[ticker_index].show()
    except:
      print("Unable to show " + str(str(row['Ticker'])))
  else:
    continue
print("Finished plotting breakout charts.")

Plotting breakout charts..


Unable to show OSIR


Unable to show IMMY


Unable to show JBX


Finished plotting breakout charts.


3. For each entry check if the stock has closed within 20% of from the high for a valid entry.

In [0]:
original_stoploss = {}
valid_entry = {}
r = 0.75
          
print("Checking for valid breakout entries...")

for ticker_index, row in df.iterrows():
  try:
    tic = ticker[ticker_index]
    try:
      high = (tic.loc[tic['Date'] == df.iloc[ticker_index].Date, 'High']).values[0]
      low = (tic.loc[tic['Date'] == df.iloc[ticker_index].Date, 'Low']).values[0]
      close = (tic.loc[tic['Date'] == df.iloc[ticker_index].Date, 'Close']).values[0]
      
      ticker_name = df.Ticker.iloc[[ticker_index]].to_string(index=False)
      signal_date = df.iloc[ticker_index].Date.date()
      if (close>=((high-low)*r)+low):
        print("Valid entry "+str(ticker_name)+" "+str(signal_date))
        original_stoploss[ticker_index] = low
        print("Stoploss: ",original_stoploss[ticker_index])
        valid_entry[ticker_index] = True
        continue
      else:
        print("No valid entry "+str(ticker_name)+" "+str(signal_date))
        valid_entry[ticker_index] = False
        print("HOD: " + str(high))
        print("LOD: " + str(low))
        print("Close: " + str(close))
        print("Percent of bar: " + str((close-low)/(high-low)*100) + "%")
    except:
      print("Could not check " + str(df.Ticker.iloc[ticker_index]))
  except:
    continue
print("Finished checking for valid breakout entries.")

Checking for valid breakout entries...
No valid entry  FLNT 2018-12-07
HOD: 4.159999847412109
LOD: 3.869999885559082
Close: 3.9600000381469727
Percent of bar: 31.0345394574579%
No valid entry  RDFN 2018-12-07
HOD: 17.940000534057617
LOD: 17.270000457763672
Close: 17.459999084472656
Percent of bar: 28.358000757245787%
Valid entry  CHD 2018-12-11
Stoploss:  66.69999694824219
Valid entry  MTLS 2018-12-12
Stoploss:  14.050000190734863
Valid entry  NEW 2018-12-12
Stoploss:  5.699999809265137
Valid entry  SSRM 2018-12-12
Stoploss:  10.850000381469727
Valid entry  AEM 2018-12-13
Stoploss:  39.029998779296875
Valid entry  AG 2018-12-17
Stoploss:  5.150000095367432
Valid entry  AUY 2018-12-17
Stoploss:  2.140000104904175
Valid entry  BVN 2018-12-17
Stoploss:  14.739999771118164
Valid entry  CDE 2018-12-17
Stoploss:  3.869999885559082
Valid entry  GDX 2018-12-17
Stoploss:  20.280000686645508
No valid entry  HI 2018-12-17
HOD: 40.029998779296875
LOD: 38.70000076293945
Close: 38.959999084472656
Pe

4. Loop through the time-series and exit based on particular criteria.

In [0]:
print("Simulating trades...")
print("Exit Strategy 1: Moving up stoploss one day at a time and letting it stop out")
stoploss = {}
equity_change = {}
equity_change_date = {}
results = 0

for ticker_index, ticker_row in df.iterrows():
  try:
    tic = ticker[ticker_index]
    stoploss[ticker_index] = original_stoploss[ticker_index]
    ticker_name = df.Ticker.iloc[[ticker_index]].to_string(index=False)
    print(ticker_name)
    #print(stoploss[ticker_index])
    if(valid_entry[ticker_index]==False):
      print('No valid entry')
      continue
    for date_index, date_row in tic.iloc[tic.loc[tic['Date'] == df.iloc[ticker_index].Date].index[0]:].iterrows():
      entry_index = tic.loc[tic['Date'] == df.iloc[ticker_index].Date].index[0]
      #print(tic.iloc[entry_index])
      date = tic.iloc[date_index].Date.date()
      high = tic['High'][date_index]
      low = ticker[ticker_index]['Low'][date_index]
      open_price = ticker[ticker_index]['Open'][date_index]
      close = ticker[ticker_index]['Close'][date_index]
      entry = ticker[ticker_index]['Close'][entry_index]
      risk = (ticker[ticker_index]['Close'][entry_index]-ticker[ticker_index]['Low'][entry_index])
      if(date_index == entry_index): 
        print('Entry date ' + str(date) + ' at ' + str(entry))
        print('Original stoploss ' + str(stoploss[ticker_index]))
        continue
      if(open_price<=stoploss[ticker_index]):
        print(date)
        print('Open at ' + str(open_price) + ' is equal to or below stoploss ' + str(stoploss[ticker_index]))
        print("stopped out at open for a " + str((open_price-entry)/entry*100) + "% change, equal to " + str((open_price-entry)/risk-1) + 'R')
        results = results + (stoploss[ticker_index]-entry)/risk
        equity_change[ticker_index] = (stoploss[ticker_index]-entry)/risk
        equity_change_date[ticker_index] = tic.iloc[date_index].Date
        break
      if(low<=stoploss[ticker_index]):
        print(date)
        print('Low of day ' + str(low) + ' is equal to or below stoploss ' + str(stoploss[ticker_index]))
        print("stopped out at stop price for a " + str((stoploss[ticker_index]-entry)/entry*100) + "% change, equal to " + str((stoploss[ticker_index]-entry)/risk) + 'R')
        results = results + (stoploss[ticker_index]-entry)/risk
        equity_change[ticker_index] = (stoploss[ticker_index]-entry)/risk
        equity_change_date[ticker_index] = tic.iloc[date_index].Date
        break
      else:
        print(date)
        print('Low of day ' + str(low) + ' is over stoploss ' + str(stoploss[ticker_index]))
        stoploss[ticker_index]=low
        print("new stoploss is now " + str(stoploss[ticker_index]))
  except:
    continue
print("Total: " + str(results) + "R")
print("Finished simulating trades.")
print(equity_change_date)
print(equity_change)

Simulating trades...
Exit Strategy 1: Moving up stoploss one day at a time and letting it stop out
 CHD
Entry date 2018-12-11 at 68.4000015258789
Original stoploss 66.69999694824219
2018-12-12
Low of day 67.44999694824219 is over stoploss 66.69999694824219
new stoploss is now 67.44999694824219
2018-12-13
Low of day 67.87999725341797 is over stoploss 67.44999694824219
new stoploss is now 67.87999725341797
2018-12-14
Low of day 68.52999877929688 is over stoploss 67.87999725341797
new stoploss is now 68.52999877929688
2018-12-17
Low of day 65.06999969482422 is equal to or below stoploss 68.52999877929688
stopped out at stop price for a 0.1900544598215904% change, equal to 0.07646876668925559R
 MTLS
Entry date 2018-12-12 at 14.949999809265137
Original stoploss 14.050000190734863
2018-12-13
Low of day 14.199999809265137 is over stoploss 14.050000190734863
new stoploss is now 14.199999809265137
2018-12-14
Low of day 14.300000190734863 is over stoploss 14.199999809265137
new stoploss is now 1

5. Visualize the results.

In [0]:
equity = {}

all_timestamps = []

for i in range(len(equity_change_date)):
  try:
    print(equity_change_date[i])
    timestamp = datetime.strptime(equity_change_date[ticker_index][i]+' '+equity_change_date['timestamp'][i],'%Y-%m-%d %H:%M:%S')
    print(timestamp)
    all_timestamps.append(timestamp)  # make a list of dates
    # Tempsfinal = max(timestamp)                 # ###
    # Tempsinitial = min(timestamp)               # move this out of loop
    # interval=int((Tempsfinal- Tempsinitial)/6)  # ### 

  except:
    print("False")

# get min/max
Tempsfinal = max(all_timestamps)
Tempsinitial = min(all_timestamps)

interval = Tempsfinal-Tempsinitial

print('Start time: ' + str(Tempsinitial))
print('End time:   ' + str(Tempsfinal))
print('interval:   ' + str(interval))

#print(type(equity_change_date))
#print(max(equity_change_date))
'''
for ticker_index, ticker_row in df.iterrows():
  try:
    #print(equity_change_date[ticker_index]==equity_change_date[ticker_index])
    #print(type(equity_change_date[ticker_index]))
    
  except:
    print("False")
    continue
'''

False
False
False
False
2018-12-17 00:00:00
False
2018-12-26 00:00:00
False
2018-12-14 00:00:00
False
2018-12-14 00:00:00
False
2018-12-14 00:00:00
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


ValueError: ignored

## **Examples**

Check that we have all necessary libraries.

In [0]:
!pip install pandas-datareader

In [0]:
import pandas_datareader as pdr
print(pdr.__version__)

0.7.4


In [0]:
import pandas as pd
print(pd.__version__)

0.24.2


In [0]:
#!pip install tensorflow

In [0]:
#import tensorflow as tf
#print(tf.__version__)

In [0]:
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.2.0
astor==0.8.0
astropy==3.0.5
atari-py==0.1.15
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.8
autograd==1.3
Babel==2.7.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
blis==0.2.4
bokeh==1.0.4
boto==2.49.0
boto3==1.9.205
botocore==1.12.205
Bottleneck==1.2.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.1
certifi==2019.6.16
cffi==1.12.3
chainer==5.4.0
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cvxopt==1.2.3
cvxpy==1.0.24
cycler==0.10.0
cymem==2.0.2
Cython==0.29.13
daft==0.0.4
dask==1.1.5
dataclasses==0.6
datascience==0.10.6
decorator==4.4.0
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.0
distributed==1.25.3
Django==2.2.4
dlib==19.16.0
dm-sonnet==1.34
docopt==0.6.2
docutils==0.14
dopamine-rl==1.0.5
easy

In [0]:
!pip install plotly==4.1.0

     |████████████████████████████████| 7.1MB 83kB/s 
  Found existing installation: plotly 3.6.1
    Uninstalling plotly-3.6.1:
      Successfully uninstalled plotly-3.6.1


In [0]:
import plotly
plotly.__version__

'4.1.0'

Example of describing data:

In [0]:
# Return first rows of `aapl`
#aapl.head()

# Return last rows of `aapl`
#aapl.tail()

# Describe `aapl`
#aapl.describe()

Example of writing to csv:

In [0]:
from google.colab import drive
import pandas as pd
drive.mount('drive')
aapl.to_csv('aapl_ohlc.csv')
!cp aapl_ohlc.csv drive/My\ Drive/
#df = pd.read_csv('data/aapl_ohlc.csv', header=0, index_col='Date', parse_dates=True)

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


Example of analyzing data frame:

In [0]:
# Inspect the index 
#aapl.index

# Inspect the columns
aapl.columns

# Select only the last 10 observations of `Close`
#ts = aapl['Close'][-10:]

# Check the type of `ts` 
#type(ts)

Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [0]:
# Inspect the first rows of November-December 2006
#print(aapl.loc[pd.Timestamp('2019-5-20'):pd.Timestamp('2019-6-20')].head())

# Inspect the first rows of 2007 
#print(aapl.loc['2007'].head())

# Inspect November 2006
#print(aapl.iloc[22:43])

# Inspect the 'Open' and 'Close' values at 2006-11-01 and 2006-12-01
#print(aapl.iloc[[22,43], [0, 3]])

Resampling to weekly or monthly:

In [0]:
# Sample 20 rows
#sample = aapl.sample(20)

# Print `sample`
#print(sample)

# Resample to monthly level 
#monthly_aapl = aapl.resample('M').mean()

# Print `monthly_aapl`
#print(monthly_aapl)

Backtesting code:

In [0]:
# Set the initial capital
initial_capital= float(100000.0)

# Create a DataFrame `positions`
positions = pd.DataFrame(index=signals.index).fillna(0.0)

# Buy a 100 shares
positions['AAPL'] = 100*signals['signal']   
  
# Initialize the portfolio with value owned   
portfolio = positions.multiply(aapl['Adj Close'], axis=0)

# Store the difference in shares owned 
pos_diff = positions.diff()

# Add `holdings` to portfolio
portfolio['holdings'] = (positions.multiply(aapl['Adj Close'], axis=0)).sum(axis=1)

# Add `cash` to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(aapl['Adj Close'], axis=0)).sum(axis=1).cumsum()   

# Add `total` to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add `returns` to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

# Print the first lines of `portfolio`
print(portfolio.head())

Visualize equity curve:

In [0]:
# Import the `pyplot` module as `plt`
import matplotlib.pyplot as plt

# Create a figure
fig = plt.figure()

ax1 = fig.add_subplot(111, ylabel='Portfolio value in $')

# Plot the equity curve in dollars
portfolio['total'].plot(ax=ax1, lw=2.)

ax1.plot(portfolio.loc[signals.positions == 1.0].index, 
         portfolio.total[signals.positions == 1.0],
         '^', markersize=10, color='m')
ax1.plot(portfolio.loc[signals.positions == -1.0].index, 
         portfolio.total[signals.positions == -1.0],
         'v', markersize=10, color='k')

# Show the plot
plt.show()

Backtesting code:

In [0]:
def initialize(context):
    context.sym = symbol('AAPL')
    context.i = 0


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.sym, 'price', 100, '1d').mean()
    long_mavg = data.history(context.sym, 'price', 300, '1d').mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.sym, 100)
    elif short_mavg < long_mavg:
        order_target(context.sym, 0)

    # Save values for later inspection
    record(AAPL=data.current(context.sym, "price"),
           short_mavg=short_mavg,
           long_mavg=long_mavg)